In [48]:
import pandas as pd
import PySimpleGUI as sg
import os.path
import pandas as pd

#set the theme for the screen/window
sg.theme("Topanga")

# file selection layout
file_list_column = [
    [
        sg.Text("2020 Data Source"),
        sg.In(size=(25,1), enable_events=True, key="-FILE-"),
        sg.FileBrowse(file_types=(("CSV Files", "*.csv"),)),
    ],
    [
        sg.Text("2021 Data Source"),
        sg.In(size=(25,1), enable_events=True, key="-FILE1-"),
        sg.FileBrowse(file_types=(("CSV Files", "*.csv"),)),
    ],
    [
        sg.Text("YTD Month"),
        sg.In(size=(25,1), enable_events=True, key="month"),
    ],
    [
        sg.Button("Convert", enable_events=True, key="convert"),
    ],
    [
        sg.ProgressBar(1, orientation='h', size=(20, 20), bar_color=('Green','Gray'), key='progress_0')
    ],
    [
        sg.ProgressBar(1, orientation='h', size=(20, 20), bar_color=('Green','Gray'), key='progress_1')
    ],
    [
        sg.Listbox(
        values=[], enable_events=True, size=(50,5),
        key="-FILE LIST-"
        )
    ],
]

layout = [
    [
        sg.Column(file_list_column),
    ]
]

window = sg.Window("File Selector", layout) 
progress_2020 = window['progress_0']
progress_2021 = window['progress_1']

In [2]:
# functions needed for program to run

def clean_csv(file_name,to_csv,ytd_months):
    '''formats invoice file by removing excess data and changing
    data types as needed
    Args:
        file_name(str): the file containing customer data
        to_csv(bool): export the file to csv
        
    Returns: 
        df_cust(pd.DataFrame): the cleaned dataframe
    '''
    # import data from csv
    df_cust = pd.read_csv(file_name,thousands=',',index_col=False)
    
    # drop rows until product # begins with 'w'
    df_cust = df_cust.loc[df_cust['Product #'].str.startswith('W', na=False)]

    # convert revenue string to int
    df_cust['Invoiced Revenue Corp'] = df_cust['Invoiced Revenue Corp'].str.replace('$','')
    df_cust['Invoiced Revenue Corp'] = df_cust['Invoiced Revenue Corp'].str.replace(',','').fillna(0).astype('int')

    # remove rows with 0 values
    df_cust = df_cust[df_cust['Invoiced Revenue Corp'] != 0].reset_index()

    month_tup = ('None','Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec')

    # convert dates to month number 
    dates_series = df_cust['Month Number'].str.partition('/')[2].fillna(0) # use month number instead of date
    
    # add column containing month numbers
    df_cust['Month'] = dates_series.astype(int)
    
    # autofill: for each empty month, fill with value from previous row
    for idx, row in df_cust.iterrows():
        if idx == 0:
            continue
        if row['Month'] == 0:
            df_cust.loc[idx,'Month'] = df_cust.loc[idx - 1,'Month']
    
    df_na_bool = pd.isna(df_cust)

    # autofill: customer name, year number, month number
    for idx, row in df_na_bool.iterrows():
        if idx == 0:
            continue
        if row['Customer Name']:
            df_cust.loc[idx,'Customer Name'] = df_cust.loc[idx - 1,'Customer Name']
        if row['Year Number']:
            df_cust.loc[idx,'Year Number'] = df_cust.loc[idx - 1,'Year Number']        
        if row['Month Number']:
            df_cust.loc[idx,'Month Number'] = df_cust.loc[idx - 1,'Month Number']
            
    # convert numbers to month names
    for i in range(len(df_cust['Month Number'])):
        dates_series[i] = month_tup[int(dates_series[i])]

    df_cust['Month Name'] = dates_series 
    
    # ensure cust id is int type
    df_cust['Cust Account ID'] = df_cust['Cust Account ID'].astype(int)

    # remove excess columns
    #df_cust = df_cust.drop(columns=['index','Unnamed: 4'])
            
    # Sort the rows of dataframe by 'Name' column
    df_cust = df_cust.sort_values(by = 'Month',ignore_index=True)
    # only keep ytd months
    df_cust = df_cust[df_cust['Month'].isin(ytd_months)]
    
    if to_csv:
        df_cust.to_csv('sales_renewal_status.csv',index=False)
    
    return df_cust

# set renewal status for each customer
renewal_status = {}

def search_renew(prod_num, cust_id, prod_series, cust_series):
    '''search for matching prod # and cust ID pair in given
    series, iterate by index. update appearance count dict 
    renewal_status
    Args:
        prod_num(str)
        cust_id(int)
        prod_series(pd.Series)
        cust_series(pd.Series)
    Returns:
        renewed(bool): whether a match was found
    '''
    count = 0
    pair = prod_num + ',' + str(cust_id)
    prev_found = renewal_status.get(pair, 0)
    
    # if renewal_status[key] >= count, continue 
    for i in range(prod_series.size):
        # if a match is found
        if prod_num == prod_series.get(i) and cust_id == cust_series.get(i):
            # if total matches found greater than those found in this function 
            if renewal_status.get(pair, 0) > count:
                count += 1 # ignore current match
            else:
                renewal_status[pair] = count + 1 # new match found, update value
                break
    return renewal_status.get(pair,0) > prev_found

In [3]:
file_list = []
invoice_2020 = ''
invoice_2021 = ''
month_ytd = ''
# event loop
while True:
    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
    # file was chosen, so set variables as selected files
    if event == "-FILE-":
        invoice_2020 = values["-FILE-"]
        try:
            # show file in list 
            file_list.append(invoice_2020.split('/')[-1])
        except:
            file_list = []    
        window["-FILE LIST-"].update(file_list)
    elif event == "-FILE1-":
        invoice_2021 = values["-FILE1-"]
        try:
            # show file in list 
            file_list.append(invoice_2021.split('/')[-1])
        except:
            file_list = []
        window["-FILE LIST-"].update(file_list)
    elif event == "month":
        month_ytd = values["month"]
    elif event == "convert":
        # months which include data from current year
        month_list = ['None','January','February','March','April','May','June','July','August','September',
                      'October','November','December']
        current_ytd_months = []

        for idx, month in enumerate(month_list):
            if month == month_ytd:
                current_ytd_months.append(str(idx))
                break
            else:
                current_ytd_months.append(str(idx))
        df_2021 = clean_csv(invoice_2021,True,current_ytd_months)
        df_2020 = clean_csv(invoice_2020,False,current_ytd_months)
        # remove months which have not occurred
        #df_2020 = df_2020[~df_2020['Month'].isin(['9','10','11','12'])]
        df_renew_2020 = pd.concat([df_2020['Product #'], df_2020['Cust Account ID']],axis=1)
        df_renew_2021 = pd.concat([df_2021['Product #'], df_2021['Cust Account ID']],axis=1)
        df_renew = pd.concat([df_renew_2020, df_renew_2021], axis=1)
        df_renew.columns = ['2020 Product #','2020 Cust Account ID','2021 Product #','2021 Cust Account ID']
        df_renew['2021 Product #'] = df_renew['2021 Product #'].fillna('None')
        df_renew['2021 Cust Account ID'] = df_renew['2021 Cust Account ID'].fillna(0).astype(int)
        renewals = []
        for idx, row in tqdm(df_renew.iterrows(), total=df_renew.shape[0]):
            progress_2020.update_bar(idx,df_renew.shape[0])
            prod_num = row['2020 Product #']
            cust_id = row['2020 Cust Account ID']
            prod_series = df_renew['2021 Product #']
            cust_series = df_renew['2021 Cust Account ID']
            if search_renew(prod_num,cust_id,prod_series,cust_series):
                renewals.append('Renewed')
            else:
                renewals.append('Not Renewed')
        if 'Renewal Status' in df_2020.columns:
            df_2020['Renewal Status'] = renewals
        else:
            df_2020.insert(2,'Renewal Status',renewals,True)

        df_2020 = df_2020.iloc[:, lambda df: [1,2,3,4,5,6,7,8,9,10,11,12,13]]

        df_2020.to_csv('2020_renewal_data.csv',index=False)
        renewed = []
        for idx, row in tqdm(df_renew.iterrows(), total=df_renew.shape[0]):
            progress_2021.update_bar(idx,df_renew.shape[0])
            if row['2021 Cust Account ID'] == 0:
                continue
            prod_num = row['2021 Product #']
            cust_id = row['2021 Cust Account ID']
            prod_series = df_renew['2020 Product #']
            cust_series = df_renew['2020 Cust Account ID']
            if search_renew(prod_num,cust_id,prod_series,cust_series):
                renewed.append('Renewed')
            else:
                renewed.append('New') 
        if 'Renewal Status' in df_2021.columns:
            df_2021['Renewal Status'] = renewed
        else:
            df_2021.insert(2,'Renewal Status',renewed,True)

        #df_2021 = df_2021.iloc[:, lambda df: [1,2,3,4,5,7,8,9,10,11,12,13,14,15,16]]

        df_2021.to_csv('2021_renewal_data.csv',index=False)
        
window.close()

df_2021 = clean_csv(invoice_2021,True)
df_2020 = clean_csv(invoice_2020,False)
# remove months which have not occurred
df_2020 = df_2020[~df_2020['Month'].isin(['9','10','11','12'])]
df_2020.head()

# form dataframe which puts 2020 and 2021 customers side by side
df_renew_2020 = pd.concat([df_2020['Product #'], df_2020['Cust Account ID']],axis=1)
df_renew_2021 = pd.concat([df_2021['Product #'], df_2021['Cust Account ID']],axis=1)
df_renew_2020.head()

# set renewal status for each customer
renewal_status = {}

def search_renew(prod_num, cust_id, prod_series, cust_series):
    '''search for matching prod # and cust ID pair in given
    series, iterate by index. update appearance count dict 
    renewal_status
    Args:
        prod_num(str)
        cust_id(int)
        prod_series(pd.Series)
        cust_series(pd.Series)
    Returns:
        renewed(bool): whether a match was found
    '''
    count = 0
    pair = prod_num + ',' + str(cust_id)
    prev_found = renewal_status.get(pair, 0)
    
    # if renewal_status[key] >= count, continue 
    for i in range(prod_series.size):
        # if a match is found
        if prod_num == prod_series.get(i) and cust_id == cust_series.get(i):
            # if total matches found greater than those found in this function 
            if renewal_status.get(pair, 0) > count:
                count += 1 # ignore current match
            else:
                renewal_status[pair] = count + 1 # new match found, update value
                break
    return renewal_status.get(pair,0) > prev_found

# format df to only contain 2020 and 2021 product# and cust ID
df_renew = pd.concat([df_renew_2020, df_renew_2021], axis=1)
df_renew.columns = ['2020 Product #','2020 Cust Account ID','2021 Product #','2021 Cust Account ID']
df_renew['2021 Product #'] = df_renew['2021 Product #'].fillna('None')
df_renew['2021 Cust Account ID'] = df_renew['2021 Cust Account ID'].fillna(0).astype(int)
df_renew.head()

renewals = []
for idx, row in tqdm(df_renew.iterrows(), total=df_renew.shape[0]):
    prod_num = row['2020 Product #']
    cust_id = row['2020 Cust Account ID']
    prod_series = df_renew['2021 Product #']
    cust_series = df_renew['2021 Cust Account ID']
    if search_renew(prod_num,cust_id,prod_series,cust_series):
        renewals.append('Renewed')
    else:
        renewals.append('Not Renewed')
renewals

'''counts_2020 = df_renew['2020 Cust Account ID'].value_counts()
counts_2021 = df_renew['2020 Product #'].value_counts()
print(counts_2020[10900])
print(counts_2021['WCAM1YR10'])'''

if 'Renewal Status' in df_2020.columns:
    df_2020['Renewal Status'] = renewals
else:
    df_2020.insert(2,'Renewal Status',renewals,True)

df_2020 = df_2020.iloc[:, lambda df: [1,2,3,4,5,6,7,8,9,10,11,12,13]]
    
df_2020.to_csv('2020_renewal_data.csv',index=False)
df_2020.head()
# 2020 data csv############################

# show the percentage of customers who renewed 
renewal_percentage = (df_2020['Renewal Status'].value_counts(normalize=True))*100
renewal_percentage

# 2021: renewed/new column
'''to add renewed column in 2021:
    reuse matching algorithm and if match is found, renewed'''
renewed = []
for idx, row in tqdm(df_renew.iterrows(), total=df_renew.shape[0]):
    if row['2021 Cust Account ID'] == 0:
        continue
    prod_num = row['2021 Product #']
    cust_id = row['2021 Cust Account ID']
    prod_series = df_renew['2020 Product #']
    cust_series = df_renew['2020 Cust Account ID']
    if search_renew(prod_num,cust_id,prod_series,cust_series):
        renewed.append('Renewed')
    else:
        renewed.append('New') 
renewed

if 'Renewal Status' in df_2021.columns:
    df_2021['Renewal Status'] = renewed
else:
    df_2021.insert(2,'Renewal Status',renewed,True)

df_2021 = df_2021.iloc[:, lambda df: [1,2,3,4,5,7,8,9,10,11,12,13,14,15,16]]

df_2021.to_csv('2021_renewal_data.csv',index=False)
df_2021.head()